<a href="https://colab.research.google.com/github/dujest/yolov5-workshop/blob/main/YOLOv5_Hyperparameter_Evolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ./drive/MyDrive/

In [ ]:
!mkdir ./colab

In [2]:
%cd /content/drive/MyDrive/colab

/content/drive/MyDrive/colab


# Install Requirements

In [3]:
# !git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

/content/drive/MyDrive/colab/yolov5
     |████████████████████████████████| 1.6 MB 4.8 MB/s 
     |████████████████████████████████| 145 kB 5.2 MB/s 
     |████████████████████████████████| 178 kB 50.9 MB/s 
     |████████████████████████████████| 1.1 MB 50.2 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 138 kB 43.2 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
Setup complete. Using torch 1.12.1+cu113 (CPU)
